In [22]:
import pickle
import pandas as pd
import sys
sys.path.append("..")
from toputils import *
from tqdm import tqdm

tgttop, _, config = read_config()
print(tgttop)

sz2nd


In [23]:
trajs = pickle.load(open(f"data/trajs_release_{tgttop}.pkl", "rb"))
print(len(trajs))
print(len({t["vid"] for t in trajs}))

1650253
1122385


In [24]:
# 生成traj.csv
columns = ["VehicleID", "TripID", "Points", "DepartureTime", "Duration", "Length"]
lines = []
cid2vid = {}
for tj in tqdm(trajs):
    cid = tj["vid"]
    if cid in cid2vid:
        vid = cid2vid[cid]
    else:
        vid = len(cid2vid)
        cid2vid[cid] = vid

    ps = tj["ps"]
    ps = [[n, round(t, 1)] for n, t in ps]
    dt = ps[0][1]
    dr = ps[-1][-1] - ps[0][1]
    ps = "_".join([
        "-".join([str(n), f"{t:.1f}"])
        for n, t in ps])
    
    lines.append([
        str(vid), str(tj["tid"]), ps, f"{dt:.1f}", f"{dr:.1f}", f"{tj['l']:.1f}"
    ])
pd.DataFrame(columns=columns, data=lines).to_csv(f"traj_{tgttop}.csv", index=False)

100%|██████████| 1650253/1650253 [05:49<00:00, 4728.05it/s] 


In [21]:
# 生成node.csv, edge.csv
G = pickle.load(open(f"../data_interface/G_{tgttop}.pkl", "rb"))
columns = ["NodeID", "Longitude", "Latitude", "HasCamera"]
lines = []
for nid, n in G.nodes(data=True):
    lines.append([
        str(nid),
        str(round(n["x"], 6)),
        str(round(n["y"], 6)),
        '1' if n.get("camera", None) else '0'
    ])
pd.DataFrame(columns=columns, data=lines).to_csv(f"node_{tgttop}.csv", index=False)
columns = ["Origin", "Destination", "Class", "Geometry", "Length"]
lines = []
for _, _, e in G.edges(data=True):
    lv = e["highway"]
    if not isinstance(lv, str):
        lv = lv[0]
    geo = "_".join([str(round(lon, 6)) + "-" + str(round(lat, 6)) for lon, lat in e["points_gps"]])
    lines.append([
        str(e["od"][0]),
        str(e["od"][1]),
        lv,
        geo,
        str(round(e["length"], 1))
    ])
pd.DataFrame(columns=columns, data=lines).to_csv(f"edge_{tgttop}.csv", index=False)